In [ ]:
import numpy as np
import skfuzzy as fuzz
import pandas as pd
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split




In [65]:
#Acesso ao dataset
music = pd.read_csv('dataset.csv', index_col=0) #chama o ficheiro e remove coluna desnecessária (index_col)

#Garantir que o acesso foi bem sucedido (faz print das primeiras 5 linhas do dataset)
print(music.head())
music.shape

                 track_id                 artists  ... time_signature track_genre
0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino  ...              4    acoustic
1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward  ...              4    acoustic
2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN  ...              4    acoustic
3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis  ...              3    acoustic
4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet  ...              4    acoustic

[5 rows x 20 columns]


(114000, 20)

meti só para teste para garantir que funciona podemos tirar no final

In [ ]:
##limpeza do dataset
# Contar quantas linhas duplicadas existem (mesmo nome + mesmo artista)
duplicadas_antes = music.duplicated(subset=["track_name", "artists"]).sum()
print(f"Antes da limpeza: {duplicadas_antes} músicas duplicadas")

# Remover duplicados (mesmo nome e mesmo artista)
music = music.drop_duplicates(subset=["track_name", "artists"], keep="first")

#valores em falta por coluna
print(music.isnull().sum()) 
music = music.dropna()  #por serem poucos


Antes da limpeza: 0 músicas duplicadas
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [ ]:
#defenir as features
f1 = music.values[:, 7]    #danceability
f2 = music.values[:, 8]    #energy
f3 = music.values[:, 9]    #key
f4 = music.values[:, 10]   #loudness
f5 = music.values[:, 11]   #mode
f6 = music.values[:, 13]   #acousticness
f7 = music.values[:, 17]   #tempo (bpm)
f8 = music.values[:, 18]   #time_signature
f9 = music['track_genre'].values   #track_genre
y = music.values[:, 16]    #valence-target

# matriz features
x = music.values[:, [7, 8, 9, 10, 11, 13, 17, 18, 19]]
#x = music[['danceability', 'energy', 'key',.....]]    

print(f1)
print(f9)
print(y)
print(x)
music.shape

[0.676 0.42 0.438 ... 0.629 0.587 0.526]
['acoustic' 'acoustic' 'acoustic' ... 'world-music' 'world-music'
 'world-music']
[0.715 0.267 0.12 ... 0.743 0.413 0.708]
[[0.676 0.461 1 ... 87.917 4 'acoustic']
 [0.42 0.166 1 ... 77.489 4 'acoustic']
 [0.438 0.359 0 ... 76.332 4 'acoustic']
 ...
 [0.629 0.329 0 ... 132.378 4 'world-music']
 [0.587 0.506 7 ... 135.96 4 'world-music']
 [0.526 0.487 1 ... 79.198 4 'world-music']]


(81344, 20)

In [ ]:
#verificaçaõ da limpeza
duplicadas_depois = music.duplicated(subset=["track_name", "artists"]).sum()
print(f"Depois da limpeza: {duplicadas_depois} músicas duplicadas")


Depois da limpeza: 0 músicas duplicadas


In [ ]:
#train and test split
test_size = 0.3
xtr, xte, ytr, yte = train_test_split(x, y, test_size = test_size, random_state = 42)

'-------------------------------------------------'
#criar 3 conjuntos???? tipo treino, teste e validação no final, sendo esse aquele que o algoritmo nunca viu